In [1]:
!pip3 install snscrape
!pip install tqdm
!pip install pmaw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB 850.6 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.6/176.6 KB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 KB 1.9 MB/s eta 0:00:00


In [1]:
import time
import numpy as np
import requests
import pandas as pd
import snscrape
from tqdm import tqdm
from pmaw import PushshiftAPI
import snscrape.modules.twitter as sntwitter
import itertools
import os

In [2]:
%config Completer.use_jedi = False

In [7]:
## REDDIT

# we can place any keywords we want in the query here
query = "quit my job"

# we then build the url using the above defined query. 
# we can also build multiple urls with different queries in this way
url = f"https://api.pushshift.io/reddit/search/comment/?q={query}&size=1000"
# get the results from this request, and put them into a dataframe
request = requests.get(url)
json_response = request.json()
temp_df = pd.DataFrame(json_response['data'])
# here i condense the dataframe a bit just for testing purposes
reddit_df = temp_df[['author', 'body', 'link_id', 'parent_id', 'subreddit']].copy()

In [8]:
reddit_df

,author,body,link_id,parent_id,subreddit
0,Dear_Effective9745,This is my entrance. I hope I win. 450m bank a...,t3_u8w1ll,t3_u8w1ll,2007scape
1,PM_ME_YOUR_FUGACITY,"Hey bud, similar situation. Wife had a bipola...",t3_u8l1rx,t1_i5oc7fb,AskReddit
2,Always_on_my_mind12,I hear you. I ended up with two failed relatio...,t3_u8nvid,t3_u8nvid,AskWomenOver30
3,oceanmanbuthere,Get more cats!! And I probably would still hav...,t3_u81fs4,t1_i5nmcjh,antiwork
4,PM_ME_YOUR_FUGACITY,This is probably it. This is my bottom.\n\nI h...,t3_u8l1rx,t3_u8l1rx,AskReddit
...,...,...,...,...,...
95,Yellowjacketnumber7,NTA. Full disclosure - I make ALOT more than m...,t3_u8pxsf,t3_u8pxsf,AmItheAsshole
96,duckwithaboater,Oh my gosh I feel for you. I tried my first jo...,t3_u7xppk,t1_i5m57d9,AskReddit
97,Tired_Beyond_Belief,The ones railing against having the Dungeon Fi...,t3_u7wyue,t1_i5jvgsi,classicwow
98,andreaak88,"My partner and I both earn similar incomes, so...",t3_u8pxsf,t3_u8pxsf,AmItheAsshole


In [11]:
## REDDIT

# we can place any keywords we want in the query here
query = "quit job"

# we then build the url using the above defined query. 
# we can also build multiple urls with different queries in this way
url = f"https://api.pushshift.io/reddit/search/comment/?q={query}&size=1000"
# get the results from this request, and put them into a dataframe
request = requests.get(url)
json_response = request.json()
temp_df2 = pd.DataFrame(json_response['data'])
# here i condense the dataframe a bit just for testing purposes
reddit_df2 = temp_df2[['author', 'body', 'link_id', 'parent_id', 'subreddit']].copy()

In [12]:
reddit_df2

,author,body,link_id,parent_id,subreddit
0,Cordath,"Well, the first thing I'd say is don't be so h...",t3_u6qlp5,t1_i5irq8o,bahai
1,the_passengerX,Lol I bet you do considering you have a job. I...,t3_u8pxsf,t1_i5mpibn,AmItheAsshole
2,xarexen,I know you don't get paid. In English we freq...,t3_pmi0rx,t1_i5lq7go,AskModerators
3,SidsNancy,Did I miss the part where you wanter to quit y...,t3_u8pxsf,t3_u8pxsf,AmItheAsshole
4,Dear_Effective9745,This is my entrance. I hope I win. 450m bank a...,t3_u8w1ll,t3_u8w1ll,2007scape
...,...,...,...,...,...
95,santoleri3,&gt; to the hardware store because it was righ...,t3_u8l1rx,t1_i5ltmt0,AskReddit
96,chocolatechipwizard,It sounds like you are in an abusive work situ...,t3_u7y7io,t1_i5ka78l,DoesAnybodyElse
97,Bunker_Beans,"I would quit my job, immediately.",t3_u8uk87,t3_u8uk87,CryptoCurrency
98,ICanBeKinder,Yeah don't quit your day job...,t3_u8r6n8,t1_i5ng3jp,agedlikemilk


In [10]:
def get_reddit_data(query):
    url = f"https://api.pushshift.io/reddit/search/comment/?q={query}"
    # get the results from this request, and put them into a dataframe
    request = requests.get(url)
    json_response = request.json()
    temp_df = pd.DataFrame(json_response['data'])
    # here i condense the dataframe a bit just for testing purposes
    reddit_df = temp_df[['author', 'body', 'link_id', 'parent_id', 'subreddit']].copy()
    write_data_to_pkl(reddit_df, query.split(", ")[0] + "_" + query.split(", ")[1] + ".pkl" )

In [11]:
import pickle as pkl
def write_data_to_pkl(df1, file_name):
    pkl_file = open(file_name, 'wb')
    pkl.dump(df1, pkl_file)
 #    gfile = drive.CreateFile({'parents': [{'id': "185FQgIc7Y6ovkBfNHMpwgQswKWJ5jMyU"}]})
 #    # Read file and set it as the content of this instance.
 #    gfile.SetContentFile(file_name)
 #    gfile.Upload() # Upload the file.
    pkl_file.close()

   
 # Open the pickle file and load in the old dataframe
def get_data_from_pkl(file_name):
    df1 = pd.read_pickle(file_name)
    return df1

In [15]:
## It takes around 27mins and 46 seconds to get 100 commnents (It will take a very long time to get around 20,000)
## This is using the pmaw library to interact with a PushshiftAPI object just a different way of getting the data. It does the same thing as Shreya's code.
api = PushshiftAPI()
subreddit = "careers"
limit = 5000 # number of comments we want to retrieve
comments = api.search_comments(q = 'quit my job', subreddit=subreddit, limit=limit, )
# posts = api.search_submissions(subreddit = subreddit, limit = limit)
print(f'Retrieved {len(comments)} comments from Pushshift')
comments_df = pd.DataFrame(comments)

Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.


Retrieved 16 comments from Pushshift


In [16]:
comments_df

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,distinguished,gilded,retrieved_utc,score_hidden,subreddit_name_prefixed,subreddit_type,steward_reports,author_created_utc,nest_level,reply_delay
0,[],NaN,dolittleGrove,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],NaN,shrewdnegotiation,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],NaN,suitable_host,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],NaN,donkeycoco,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],NaN,haytch123456,NaN,None,[],NaN,None,NaN,text,...,NaN,0.0,1.645076e+09,False,r/Careers,public,NaN,NaN,NaN,NaN
5,[],NaN,trust_me_on_that_one,NaN,None,[],NaN,None,NaN,text,...,NaN,0.0,1.641316e+09,False,r/Careers,public,NaN,NaN,NaN,NaN
6,[],NaN,gogo--yubari,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,[],NaN,kitsinni,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,[],NaN,BlackKleenexBox,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,[],NaN,0xnss,NaN,None,[],NaN,None,NaN,text,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
## REDDIT

# we can place any keywords we want in the query here
query = "quit my job"
# we then build the url using the above defined query. 
# we can also build multiple urls with different queries in this way
url = f"https://api.pushshift.io/reddit/search/comment/?q={query}"
# get the results from this request, and put them into a dataframe
request = requests.get(url)
json_response = request.json()
temp_df2 = pd.DataFrame(json_response['data'])
# here i condense the dataframe a bit just for testing purposes
reddit_df2 = temp_df2[['author', 'body', 'link_id', 'parent_id', 'subreddit']].copy()

In [14]:
reddit_df2

,author,body,link_id,parent_id,subreddit
0,carrotwax,My brother has pretty strong aspergers and lov...,t3_tfdaci,t1_i0vaf4w,ottawa
1,Ichthyologist,"I quit a ""dream job"" to hike the AT and was un...",t3_tfg2av,t3_tfg2av,AppalachianTrail
2,yuckscott,I had a job in technical sales and was one of ...,t3_tffwtg,t3_tffwtg,AskEngineers
3,Mandouie,I worked at the Starbucks ran by OttawaU. They...,t3_tfdaci,t3_tfdaci,ottawa
4,seamonkey420,"yea, i def need to get one. i had funded a 3d ...",t3_te0e6o,t1_i0vqr2k,GPDPocket
5,Critical_Garbage_119,Your experience is likely far more common than...,t3_tfg2av,t3_tfg2av,AppalachianTrail
6,professorbix,Definitely true. But it's just odd that a 1%er...,t3_tep6by,t1_i0vm1dr,TrueOffMyChest
7,Mothman-will-rise,I could finally quit my job and start my own b...,t3_tekz8s,t3_tekz8s,WouldYouRather
8,depressedcunt69,Okay I'm very happy for you regarding the turn...,t3_ter7ci,t1_i0top8n,petsmart
9,throwawayward72,I worked retail so I quit immediately without ...,t3_tfgh7p,t1_i0vnxk6,AsOneAfterInfidelity


In [48]:
# date 
def get_reddit_comments(query, before_date):
    comment_url = f"https://api.pushshift.io/reddit/search/comment/?q={query}&size=1000&before="
    comment_url+=before_date
    request = requests.get(comment_url)
    json_response = request.json()
    temp_df = pd.DataFrame(json_response['data'])
    temp_df = temp_df[['author', 'body', 'link_id', 'parent_id', 'subreddit']].copy()
    return temp_df

In [41]:
def add_to_main_df(main_df, df_list):
    main_df = pd.concat(main_df, temp_df)
    return mai_df

In [49]:
test = get_reddit_comments("quit job", "45m")
test

,author,body,link_id,parent_id,subreddit
0,PanthersJB83,Yeah it did. I quit and found better working c...,t3_u8vt7v,t1_i5o705y,EDH
1,PM_ME_YOUR_FUGACITY,This is probably it. This is my bottom.\n\nI h...,t3_u8l1rx,t3_u8l1rx,AskReddit
2,Successful_Hunter_84,"i just quit this job and shift in SA, TX. orie...",t3_u8z2g0,t3_u8z2g0,UPS
3,Normal_Total,I'd get an operation on my feet so I could run...,t3_u8uk87,t3_u8uk87,CryptoCurrency
4,Swoozibootz,NTA - He's not a husband. He's acting like he...,t3_u8u6mx,t3_u8u6mx,AmItheAsshole
...,...,...,...,...,...
95,drewrykroeker,It really depends what kind of job environment...,t3_u8a3cg,t3_u8a3cg,jobs
96,Jumpy_Spend_5434,Yes exactly. I realized he had this issue at w...,t3_u8jj3f,t1_i5o0igi,Bumble
97,Majestic-Quit-671,"Wow, not your job to educate him on the intern...",t3_u8q0mq,t1_i5mvgo9,ireland
98,phatkidd76,"Work at a pizza shop for like a year, 90% of t...",t3_u8xdqa,t3_u8xdqa,AskReddit


In [36]:
test2 = get_reddit_comments("quit job")
test2

,author,body,link_id,parent_id,subreddit
0,GaryBot-GPT2,I'd like to know where you stand.\n\nNot sure ...,t3_u8xns2,t1_i5o270c,SubSimGPT2Interactive
1,crappy-pete,No other debts. Worked a job for 3.5 years wit...,t3_u8k3z0,t3_u8k3z0,AusFinance
2,JbearNV,Yep. I worked as a temp drafter for Nvenergy 1...,t3_u8wgeb,t1_i5ny6l4,Reno
3,serena999,"This wasn’t a formal job offer yet, it was a t...",t3_u8z9e3,t3_u8z9e3,AmItheAsshole
4,SuspiciousMallow,NTA\n\n💯 do NOT quit your job. He wants contro...,t3_u8pxsf,t3_u8pxsf,AmItheAsshole
...,...,...,...,...,...
95,CheyenneOU812,its a fucking job not jail. ill quit when i d...,t3_u8xs63,t3_u8xs63,thatHappened
96,lulu1234567777,Don’t quit your job!! But suggest that you eac...,t3_u8pxsf,t3_u8pxsf,AmItheAsshole
97,Jawato44,"NTA Do not quit your job, he wants a free ma...",t3_u8pxsf,t3_u8pxsf,AmItheAsshole
98,LilBobcat_88,&gt; It’s not my job to figure them out. And i...,t3_u8rzz0,t1_i5nymuz,alberta


In [37]:
test3 = get_reddit_comments("quit my job")
test3

,author,body,link_id,parent_id,subreddit
0,AdamJadam,One of my friends was an EMT making $15 an hou...,t3_u8lvat,t1_i5obr4m,antiwork
1,JbearNV,Yep. I worked as a temp drafter for Nvenergy 1...,t3_u8wgeb,t1_i5ny6l4,Reno
2,Environmental_Leg179,Congratulations. My wife just gave her two wee...,t3_u8z7ua,t3_u8z7ua,antiwork
3,MunzO84,Was addicted on Lyrica ( pregabalin) for like ...,t3_u8l1rx,t3_u8l1rx,AskReddit
4,dkfemspi,"First and foremost, I don't think you are abus...",t3_u80fy9,t1_i5k130d,AmItheAsshole
...,...,...,...,...,...
95,wageslavewealth,"I tried taking a week off, which was really ni...",t3_ssi77d,t1_i5neuta,covidlonghaulers
96,cashclubatl,I quit that job in February Lmao. Started maki...,t3_u8emfn,t1_i5nvqgk,Chiraqology
97,fixin2wander,I'm not a SAHM parent either and happily send ...,t3_u82e9h,t1_i5iif7r,FIREyFemmes
98,sneakpeekbot,Here's a sneak peek of /r/leanfire using the [...,t3_u8v281,t1_i5nw62x,PersonalFinanceCanada


In [42]:
test4 = test2.merge(test3,)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
test2.drop_dup

In [50]:
def create_windows(integer_val, time_val):
    windows = list(range(integer_val))
    windows = [str(x)+time_val for x in windows]
    return windows

In [77]:
def get_reddit_comments_pipe(queries, window_range, time_val, sub_or_com = ["comment", "submission"]):
    q_dataframes = []
    for q in queries:
        before_date_list = create_windows(window_range, time_val)
        
        for i in range(len(before_date_list)):
            temp_df = get_reddit_comments(query, before_date_list[i])
            q_dataframes.append(temp_df)
            
    result = pd.concat(q_dataframes)
    result.drop_duplicates(inplace=True)
    return result

    

In [68]:
queries = ['lose job', 'leave job']

In [72]:
items = '''"quit my job" "left my job" "resigned from my job" "switched careers" "changed careers" "switching careers" "changing careers" "pursuing a new career "pursue different career options" "which career should I choose" "looking for a new job" "looking for a different career" "thinking about quitting my job "may be quitting my job" "considering quitting my job" "they quit their job" "switch careers" "change careers"'''
q2 = items.split("\" \"")
len(q2)


16

In [73]:
q2comma = ", ".join([x for x in q2])

In [75]:
q2comma = q2comma[1:-2]
q2comma

'quit my job, left my job, resigned from my job, switched careers, changed careers, switching careers, changing careers, pursuing a new career "pursue different career options, which career should I choose, looking for a new job, looking for a different career, thinking about quitting my job "may be quitting my job, considering quitting my job, they quit their job, switch careers, change career'

In [76]:
query_list = q2comma.split(", ")
query_list

['quit my job',
 'left my job',
 'resigned from my job',
 'switched careers',
 'changed careers',
 'switching careers',
 'changing careers',
 'pursuing a new career "pursue different career options',
 'which career should I choose',
 'looking for a new job',
 'looking for a different career',
 'thinking about quitting my job "may be quitting my job',
 'considering quitting my job',
 'they quit their job',
 'switch careers',
 'change career']

In [80]:
from time import time
start = time()
tdf = get_reddit_comments_pipe(query_list, 2, "d")
end = time()
runtime = round(end- start, 2)
runtime = runtime/60
print("Runtime was ", runtime, " minutes!")

Runtime was  0.791  minutes!


In [79]:
tdf

,author,body,link_id,parent_id,subreddit
0,thechozenwan,Crappy posts make it easy to quit but the cush...,t3_u8vew3,t3_u8vew3,securityguards
1,Joe-Burly,Quit. Can you call the other job?,t3_u8m3ew,t3_u8m3ew,antiwork
2,rainonatent,I quit my cash job right before dirt season. S...,t3_u8jsef,t1_i5md8n0,retailhell
3,TwirlingSquirrel,I hid in the women’s restroom at Golden Corral...,t3_u8l1rx,t3_u8l1rx,AskReddit
4,OL2052,As a personal shopper I completely get it. My ...,t3_u8prje,t3_u8prje,walmart
...,...,...,...,...,...
0,BoytoyCowboy,"Once again, I can afford the credit card dept....",t3_u32w60,t1_i4mvej2,personalfinance
0,consumervigilante,Great question. I enjoy seeing the responses. ...,t3_u90jke,t3_u90jke,jobs
0,MISTERDIEABETIC,Friend helped me get a job at the company he w...,t3_u4a8eb,t3_u4a8eb,AskReddit
0,hnsl27,"Well, I’m worried sick now. Tuesday is my most...",t3_u26m0c,t3_u26m0c,infertility


In [ ]:
tdf.to_csv("data1_reddit.csv")

In [58]:
x = [1,2,3]
y = [4,5,6]
x.append(y)
x

[1, 2, 3, [4, 5, 6]]

In [53]:
pipeddata = get_reddit_comments_pipe(query, 100, "d")
pipeddata

,author,body,link_id,parent_id,subreddit
0,hxnnydew,exactly! i myself just quit yesterday after 2 ...,t3_u8zhfi,t1_i5oi2bb,CVS
1,Lutrijk,Don't quit your day job.,t3_u8vo80,t1_i5oa83z,todayilearned
2,transcribot,"Jesus: ""Do not worry""\n\nMe: ""0k, but what if ...",t3_u90736,t1_i5ohugg,TranscribersOfReddit
3,Affectionate-Bee5745,The only problem with that is that I live in a...,t3_u8b3i6,t1_i5od5x6,Lowes
4,Fearlessleader85,I can't claim to know the context of all of th...,t3_u8vf74,t1_i5oesvi,AskReddit
...,...,...,...,...,...
95,alt_spaceghoti,I don't know if you're lying or just ignorant....,t3_s2g6sq,t1_hsenlmz,AmericanPolitics
96,OFFRIMITS,"Yup, you two have hit the ""ceiling"" in terms o...",t3_s2buac,t3_s2buac,antiwork
97,mountain_Dweller,Your concerns are 100% valid and your employer...,t3_s2f945,t3_s2f945,internetparents
98,ChronicallyHungry,Lol. Military support for rent will def not co...,t3_s2aha5,t1_hsei2i7,latvia


In [54]:
pipeddata.to_csv("quit_my_job_100.csv")

In [45]:
pipeddata.drop_duplicates(inplace=True)
pipeddata

,author,body,link_id,parent_id,subreddit
0,Competitive-Meat4022,I recently quit a job after finishing the seco...,t3_u8yyif,t3_u8yyif,antiwork
1,AdMundane4716,LMFAOOOOO!!!! And Cynthia Smith is threatening...,t3_u8yt1b,t3_u8yt1b,CelebWivesofNashville
2,Odd-Employer-5529,I quit a job in under 30 min once. IF you know...,t3_u8yyif,t3_u8yyif,antiwork
3,bbmunoz,Girl leave him. Marriage is a partnership and ...,t3_u8zdd8,t3_u8zdd8,relationship_advice
4,kchizz,Not really though. Unless you quit some high l...,t3_u8e6mk,t1_i5n6ybl,antiwork
...,...,...,...,...,...
97,wxldflxwr,I've definitely still got a lot to learn. I te...,t3_u8vpld,t1_i5o5v5k,antiwork
98,justifications,Don't sit idly by a company with practices lik...,t3_u8rmp3,t1_i5myt6b,gamedev
99,davis63,That is a must he needs a job if you let him m...,t3_u8xuje,t3_u8xuje,AmItheAsshole
0,Special-Tough-3752,NTA Quitting your job will make you financiall...,t3_u8pxsf,t3_u8pxsf,AmItheAsshole


In [51]:
windows = create_windows(730, "d")
windows[0]

'0d'